In [2]:
"""
TODO: 
+ https://adeshpande3.github.io/Applying-Machine-Learning-to-March-Madness

# add to dfDiffVec 
dfSeasonResultsDet["Pred"] = [1.0] * len(dfSeasonResultsDet.iloc[:, 0])

"""

# import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [36]:
# load interim datasets
# dfTeamNames = pd.read_csv("../data/raw/WTeams.csv")
# dfSeasonInfo = pd.read_csv("../data/raw/WSeasons.csv")

dfSeasonResultsCom = pd.read_csv("../data/raw/WRegularSeasonCompactResults.csv")
# dfTourneyResultsCom = pd.read_csv("../data/raw/WNCAATourneyDetailedResults.csv")

dfStats = pd.read_csv("../data/interim/TeamStats.csv")
dfSeasonResultsCom.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


## Preparation: Team Vectors & Team Vector Difference
* load data file instead of running every time
* load regular season compact results and for each matchup, find each teams stats from the dfStats and subtract them then save in df and output to file

In [45]:
# get the team stats vector given season & id number
# made global because may want to be called manually at some point
def getTeamVec(dfStats, Season, TeamID): 
    dfReduced = dfStats.loc[(dfStats["Season"]==Season) & (dfStats["TeamID"]==TeamID)]
    dfTeamVec = dfReduced.iloc[:,2:]
    return dfTeamVec





df1 = getTeamVec(dfStats, 2014, 3108)
df1.reset_index(drop = True, inplace = True)
# df1["col2"] = [42]
print(df1)
df2 = getTeamVec(dfStats, 2014, 3120)
df2.reset_index(drop = True, inplace = True)
# df2["col2"] = [37]
print(df2)

diff = df1.subtract(df2)
print(diff)



   WinRatio
0     0.267
   WinRatio
0     0.533
   WinRatio
0    -0.266


In [53]:
def makeDiffVec(dfStats, dfSeasonMatchups):
    
    # create dfDiffVec with required columns of dfStats & such
    dfDiffVec = dfSeasonMatchups.copy().drop(columns = ["DayNum", "WScore", "LScore", "WLoc", "NumOT"], axis = 1)
    dfDiffVec.rename(columns = {'WTeamID': 'Team1', 'LTeamID': 'Team2'}, inplace = True)
    colToAdd = dfStats.columns[2:]
    for col in colToAdd:
        dfDiffVec[col] = 0
        
    print(dfDiffVec)

    for index, row in dfSeasonMatchups.iterrows():
        
        # get teamStatsDf
        dfTeamVecW = getTeamVec(dfStats, row["Season"], row["WTeamID"])
        dfTeamVecL = getTeamVec(dfStats, row["Season"], row["LTeamID"])
        
        # reset index so they can be subtracted & subtract
        dfTeamVecW.reset_index(drop = True, inplace = True)
        dfTeamVecL.reset_index(drop = True, inplace = True)
        dfTeamDiffVec = dfTeamVecW.subtract(dfTeamVecL)
        
        print(dfTeamDiffVec)
        
        dfDiffVec.iloc[index,:] = pd.concat([dfDiffVec.iloc[index,:], dfTeamDiffVec], axis=1)

        
    return dfDiffVec
    
seasons = list(range(2014,2019))
dfDiffVec = makeDiffVec(dfStats, dfSeasonResultsCom.loc[(dfSeasonResultsCom["Season"]==seasons[0]) 
                                                           | (dfSeasonResultsCom["Season"]==seasons[1]) 
                                                           | (dfSeasonResultsCom["Season"]==seasons[2])
                                                           | (dfSeasonResultsCom["Season"]==seasons[3])
                                                           | (dfSeasonResultsCom["Season"]==seasons[4])])


# dfVectors = pd.DataFrame([diffVec, winVec]).transpose()
# dfVectors.columns = ["DiffVec","WinVec"]
# dfVectors.head()
# dfVectors.to_csv("../data/interim/TrainingVectors.csv", index=None, header=True)

        Season  Team1  Team2  WinRatio
75798     2014   3103   3194         0
75799     2014   3107   3373         0
75800     2014   3116   3358         0
75801     2014   3132   3276         0
75802     2014   3133   3228         0
75803     2014   3138   3127         0
75804     2014   3139   3156         0
75805     2014   3140   3355         0
75806     2014   3143   3253         0
75807     2014   3145   3178         0
75808     2014   3146   3108         0
75809     2014   3150   3251         0
75810     2014   3151   3104         0
75811     2014   3158   3244         0
75812     2014   3162   3254         0
75813     2014   3165   3159         0
75814     2014   3170   3169         0
75815     2014   3172   3155         0
75816     2014   3174   3348         0
75817     2014   3177   3217         0
75818     2014   3179   3166         0
75819     2014   3182   3142         0
75820     2014   3184   3275         0
75821     2014   3191   3183         0
75822     2014   3193   3

IndexError: single positional indexer is out-of-bounds

## Model

In [ ]:
def MatchupDf()

In [13]:
# feature selection
x = diffVec
# target
y = winVec
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 6)

logReg = LogisticRegression()
logReg.fit(xTrain, yTrain)
pred = logisticRegr.predict(xTest)
score = logisticRegr.score(xTest, yTest)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [5]:
# read in possible matchup of teams needed to predict
# ex. 2014_3103_3107, 2014_3103_3113, 2014_3103_3119
dfPred = pd.read_csv('../data/raw/WSampleSubmissionStage1.csv')
dfPred["Season"] = dfPred["ID"].apply(lambda x: x[0:4]).astype(int)
dfPred["Team1"] = dfPred["ID"].apply(lambda x: x[5:9]).astype(int)
dfPred["Team2"] = dfPred["ID"].apply(lambda x: x[10:14]).astype(int)
dfPred = dfPred[["ID","Season","Team1","Team2","Pred"]]
dfPred.head()
dfPred.to_csv("../data/interim/TeamPred.csv", index=None, header=True)